## Machine Learning–Assisted Screening Workflow

Following manual screening of 500 records, a supervised machine learning classifier was trained using LightGBM. The process included:

1. Preprocessing the title and abstract text using TF-IDF vectorization.
2. Training a binary classifier to distinguish eligible from ineligible studies, based on manually labeled training data.
3. Hyperparameter tuning with Optuna to optimize model performance, with emphasis on high recall.
4. Applying the trained model to the remaining records to rank them by predicted relevance.
5. Exporting prediction scores for further manual review.

In [ ]:
#Install necessary library
pip install bibtexparser
pip install pandas

In [ ]:
import bibtexparser
import pandas as pd

In [ ]:
import os

# Confirm current directory
current_directory = os.getcwd()
print(current_directory)

In [ ]:
#Define the new directory path
new_directory = os.path.join(current_directory, 'Python', 'Torayyan')

# Change the current working directory to the new directory
os.chdir(new_directory)

In [ ]:
# Confirm current directory
current_directory = os.getcwd()
print(current_directory)

In [ ]:
#中断して再開する場合はcsvにしておかないかぎり毎回ここからになるよう
with open('My Collection.bib', encoding = 'utf-8') as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

# bib_database.entries でエントリ一覧が取得できる
print("BibTeXに含まれるエントリ数:")
print(len(bib_database.entries))

In [ ]:
import pickle
import os
import bibtexparser

# 1. BibTeXデータベースを保存する
def save_bibdatabase(bib_database, filename='bibdatabase.pickle'):
    """
    BibTeXデータベースをpickleファイルとして保存
    """
    with open(filename, 'wb') as f:
        pickle.dump(bib_database, f)
    print(f"BibTeXデータベースを {filename} に保存しました")

# 2. 保存したBibTeXデータベースを読み込む
def load_bibdatabase(filename='bibdatabase.pickle'):
    """
    保存したBibTeXデータベースを読み込む
    """
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            bib_database = pickle.load(f)
        print(f"{filename} からBibTeXデータベースを読み込みました")
        print(f"エントリ数: {len(bib_database.entries)}")
        return bib_database
    else:
        print(f"保存されたファイル {filename} が見つかりません")
        return None

# 使い方の例

# 1. BibTeXファイルを最初に読み込んだ後、保存する
#with open('My Collection.bib', encoding='utf-8') as bibtex_file:
#    bib_database = bibtexparser.load(bibtex_file)
#print(f"BibTeXに含まれるエントリ数: {len(bib_database.entries)}")

# データベースを保存
save_bibdatabase(bib_database)

# 2. 次回のセッションでは、pickleファイルから直接読み込む
# bib_database = load_bibdatabase()

# 必要に応じて再度BibTeXファイルから読み込む場合
# with open('My Collection.bib', encoding='utf-8') as bibtex_file:
#     bib_database = bibtexparser.load(bibtex_file)

In [ ]:
import random
import csv
import pandas as pd

# ランダムに50件を抽出
random_entries = random.sample(bib_database.entries, 50)

In [ ]:
def format_entry(entry):
    # 各フィールドを取得（存在しない場合は空文字を返す）
    title = entry.get('title', '').strip('{}')
    
    # 著者の整形（複数著者は and で区切られている）
    authors = entry.get('author', '').replace('\n', ' ')
    authors = authors.strip('{}')
    
    # 掲載誌/会議名
    journal = entry.get('journal', entry.get('booktitle', ''))
    
    # 出版年
    year = entry.get('year', '')
    
    # DOI
    doi = entry.get('doi', '')
    
    # Abstract
    abstract = entry.get('abstract', '').strip('{}')
    
    return {
        'Title': title,
        'Authors': authors,
        'Abstract': abstract,
        'Journal': journal,
        'Year': year,
        'DOI': doi
    }

# 全エントリを変換
rayyan_entries = [format_entry(entry) for entry in random_entries]

# DataFrameに変換
df = pd.DataFrame(rayyan_entries)

# CSVファイルとして保存
output_file = 'rayyan_import.csv'
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"50件のエントリを {output_file} に保存しました。")

# 最初の数件を表示して確認
print("\n最初の3件のエントリ:")
print(df.head(3))

In [ ]:
# エントリのハッシュを計算する関数
import json
import hashlib
import re

def get_entry_hash(entry):
    entry_str = json.dumps(entry, sort_keys=True)
    return hashlib.md5(entry_str.encode()).hexdigest()

# 既に抽出したエントリのハッシュを取得
# random_entriesとrandom_entries_500の両方
already_extracted_hashes = set()

# 最初の50件のハッシュを追加
try:
    for entry in random_entries:
        already_extracted_hashes.add(get_entry_hash(entry))
    print(f"50件のエントリのハッシュを追加しました")
except NameError:
    print("random_entriesが見つかりません")

# 残りのエントリを取得
remaining_entries = []
for entry in bib_database.entries:
    entry_hash = get_entry_hash(entry)
    if entry_hash not in already_extracted_hashes:
        remaining_entries.append(entry)

print(f"残りのエントリ数: {len(remaining_entries)}")

# 残りのエントリから500件をランダムに抽出 
sample_size = min(500, len(remaining_entries))
random_entries_500 = random.sample(remaining_entries, sample_size)

# 次の500件のハッシュを追加
try:
    for entry in random_entries_500:
        already_extracted_hashes.add(get_entry_hash(entry))
    print(f"500件のエントリのハッシュを追加しました")
except NameError:
    print("random_entries_500が見つかりません")

# 抽出済みのハッシュの数を確認
print(f"抽出済みのエントリ数: {len(already_extracted_hashes)}")

# Rayyan用のデータを整形する関数（PMIDの検索機能を含む）
def format_entry(entry):
    # 各フィールドを取得（存在しない場合は空文字を返す）
    title = entry.get('title', '').strip('{}')
    
    # 著者の整形（複数著者は and で区切られている）
    authors = entry.get('author', '').replace('\n', ' ')
    authors = authors.strip('{}')
    
    # 掲載誌/会議名
    journal = entry.get('journal', entry.get('booktitle', ''))
    
    # 出版年
    year = entry.get('year', '')
    
    # DOI
    doi = entry.get('doi', '')
    
    # Abstract
    abstract = entry.get('abstract', '').strip('{}')
    
    # PMID - まず直接pmidフィールドを探し、なければnoteフィールド内を検索
    pmid = entry.get('pmid', '')
    if not pmid and 'note' in entry:
        note = entry['note']
        # noteフィールド内でPMIDを探す (例: "PMID: 12345678" または "pmid=12345678" など)
        pmid_match = re.search(r'(?:PMID|pmid)[:\s=]+(\d+)', note)
        if pmid_match:
            pmid = pmid_match.group(1)
    
    return {
        'Title': title,
        'Authors': authors,
        'Abstract': abstract,
        'Journal': journal,
        'Year': year,
        'DOI': doi,
        'PMID': pmid
    }
# 500件サンプルをRayyan用CSVに書き出し


# 残りの全エントリを変換（時間がかかる可能性あり）
print("残りのエントリをDataFrameに変換中...")
all_remaining_entries = [format_entry(entry) for entry in remaining_entries]

# DataFrameに変換
remaining_df = pd.DataFrame(all_remaining_entries)

# 基本的な統計情報を表示
print("\nDataFrameの基本情報:")
print(remaining_df.info())

print("\n欠損値の数:")
print(remaining_df.isnull().sum())

print("\n最初の3件のエントリ:")
print(remaining_df.head(3))

# CSV保存オプション（大きいファイルになる可能性があるのでコメントアウト）
# remaining_df.to_csv('remaining_entries.csv', index=False, encoding='utf-8')
# print("残りのエントリをremaining_entries.csvに保存しました")

# タイトルと抄録を組み合わせた新しい列'tiab'を作成（機械学習用）
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

print("機械学習用のtiab列を生成中...")
remaining_df['tiab'] = remaining_df.apply(create_json_text, axis=1)

print("変換完了！")

In [ ]:
import pandas as pd
import bibtexparser
import hashlib
import json
import re

# CSVファイルを読み込む
print("CSVファイルを読み込み中...")
df_50 = pd.read_csv('rayyan_import_trial50.csv')
df_500 = pd.read_csv('rayyan_import_1stml_500.csv')

print(f"50件のCSVには {len(df_50)} 行あります")
print(f"500件のCSVには {len(df_500)} 行あります")

# BibTeXファイルを読み込む（まだ読み込んでいない場合）
try:
    print(f"既存のBibTeXデータベース（{len(bib_database.entries)}件）を使用します")
except NameError:
    print("BibTeXファイルを読み込み中...")
    with open('My Collection.bib', encoding='utf-8') as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    print(f"BibTeXデータベースを読み込みました（{len(bib_database.entries)}件）")

# エントリのハッシュを計算する関数
def get_entry_hash(entry):
    entry_str = json.dumps(entry, sort_keys=True)
    return hashlib.md5(entry_str.encode()).hexdigest()

# タイトルと著者でBibTeXエントリを検索する関数
def find_bibtex_entry(title, authors, abstract=None):
    matches = []
    
    for entry in bib_database.entries:
        entry_title = entry.get('title', '').strip('{}').lower()
        entry_authors = entry.get('author', '').replace('\n', ' ').strip('{}').lower()
        
        # タイトルが一致するかチェック
        if title.lower() in entry_title or entry_title in title.lower():
            # 著者が部分一致するかチェック
            if authors.lower() in entry_authors or entry_authors in authors.lower():
                matches.append(entry)
    
    # 一致するものが複数ある場合はアブストラクトで絞り込む
    if len(matches) > 1 and abstract:
        abstract = str(abstract).lower()
        for entry in matches:
            entry_abstract = entry.get('abstract', '').strip('{}').lower()
            if abstract in entry_abstract or entry_abstract in abstract:
                return entry
    
    # 一致するものがある場合は最初のものを返す
    if matches:
        return matches[0]
    
    return None

# CSVからBibTeXエントリを復元
def restore_entries_from_csv(df):
    restored_entries = []
    not_found = 0
    
    for idx, row in df.iterrows():
        title = str(row['Title'])
        authors = str(row['Authors'])
        abstract = str(row['Abstract']) if 'Abstract' in df.columns else None
        
        entry = find_bibtex_entry(title, authors, abstract)
        if entry:
            restored_entries.append(entry)
        else:
            not_found += 1
            print(f"警告: エントリが見つかりません: {title}")
    
    print(f"復元されたエントリ: {len(restored_entries)}, 見つからなかったエントリ: {not_found}")
    return restored_entries

# CSVから50件と500件のエントリを復元
print("\n50件のエントリを復元中...")
random_entries = restore_entries_from_csv(df_50)

print("\n500件のエントリを復元中...")
random_entries_500 = restore_entries_from_csv(df_500)

# 復元されたエントリのハッシュを作成
already_extracted_hashes = set()
for entry in random_entries:
    already_extracted_hashes.add(get_entry_hash(entry))
for entry in random_entries_500:
    already_extracted_hashes.add(get_entry_hash(entry))

print(f"\n復元された抽出済みエントリの合計: {len(already_extracted_hashes)}")

# 残りのエントリを取得
remaining_entries = []
for entry in bib_database.entries:
    entry_hash = get_entry_hash(entry)
    if entry_hash not in already_extracted_hashes:
        remaining_entries.append(entry)

print(f"残りのエントリ数: {len(remaining_entries)}")

# 結果確認用に最初のいくつかのエントリを表示
if random_entries:
    print("\n復元された最初のエントリのタイトル (50件):")
    print(random_entries[0].get('title', '').strip('{}'))

if random_entries_500:
    print("\n復元された最初のエントリのタイトル (500件):")
    print(random_entries_500[0].get('title', '').strip('{}'))

In [ ]:
pip install pydantic

In [ ]:
pip install scikit-learn

In [ ]:
pip install seaborn

In [ ]:
pip install openai

In [ ]:
pip install optuna

In [ ]:
pip install lightgbm

In [ ]:
import os
import json
import time
import glob
from typing import TypeVar, Type, List, Optional
from dataclasses import dataclass
from pydantic import BaseModel
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from openai import AzureOpenAI
from tqdm import tqdm
from typing import List, Dict, Optional, Any, Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import optuna
from lightgbm import LGBMClassifier

In [ ]:
# Load the CSV file into a DataFrame
rayyan_decided_df = pd.read_csv('rayyan_decided_1st500.csv')

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(rayyan_decided_df.head())

In [ ]:
# rayyan_decided_dfに対してincludedフラグを作成
# 'notes'列に'Included'という文字列が含まれていればTrue
rayyan_decided_df['included'] = rayyan_decided_df['notes'].astype(str).str.contains('Included')

# 結果の集計を表示
print(rayyan_decided_df['included'].value_counts())

# 不整合（'included'がTrueなのに'notes'に'Excluded'が含まれる）の件数
inconsistency_count = rayyan_decided_df[rayyan_decided_df["included"] == True]["notes"].str.contains("Excluded").sum()
print(f'不整合の件数: {inconsistency_count}')

# タイトルと抄録を組み合わせた新しい列'tiab'を作成
def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

# 'tiab'列を作成
rayyan_decided_df['tiab'] = rayyan_decided_df.apply(create_json_text, axis=1)

# 最初の数行を表示して確認
print("\n最初の3行のデータ:")
print(rayyan_decided_df[['title', 'abstract', 'notes', 'included', 'tiab']].head(3))

# 必要に応じてCSVとして保存
# rayyan_decided_df.to_csv('rayyan_processed.csv', index=False)

In [ ]:
def prepare_text_data(df, text_column='tiab', label_column='included'):
    """テキストデータの前処理とTF-IDF変換"""
    texts = df[text_column].fillna('')

    vectorizer = TfidfVectorizer(
        max_features=10000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )

    X_vec = vectorizer.fit_transform(texts)
    y = df[label_column]

    class_weights = dict(zip(
        y.unique(),
        [1 / (len(y) * (y == label).mean()) for label in y.unique()]
    ))

    return X_vec, y, vectorizer, class_weights

def fbeta_score_custom(y_true, y_pred, beta=1):
    """カスタムF-betaスコアの計算"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    if precision == 0.0 and recall == 0.0:
        return 0.0
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    return fbeta

def calculate_detailed_metrics(y_true, y_pred):
    """詳細な評価指標の計算と表示"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    print("\n=== 詳細な評価指標 ===")
    print(f"感度 (Sensitivity/Recall): {sensitivity:.3f}")
    print(f"特異度 (Specificity): {specificity:.3f}")
    print(f"適合率 (Precision): {precision:.3f}")
    print(f"F1スコア: {f1:.3f}")

    print("\n=== 混同行列 ===")
    print("                  Predicted")
    print("                  Negative  Positive")
    print(f"Actual Negative    {tn:^8} {fp:^8}")
    print(f"      Positive    {fn:^8} {tp:^8}")

    return {
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'f1': f1,
        'confusion_matrix': {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp}
    }

def evaluate_model(X_vec, y, params, threshold, class_weights, beta=2):
    """モデルの評価（交差検証）"""
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    all_predictions = []
    all_true_values = []

    for train_idx, val_idx in skf.split(X_vec, y):
        X_tr = X_vec[train_idx]
        X_val = X_vec[val_idx]
        y_tr = y.iloc[train_idx]
        y_val = y.iloc[val_idx]

        model = LGBMClassifier(
            **params,
            random_state=42,
            class_weight=class_weights
        )

        model.fit(X_tr, y_tr)
        y_prob = model.predict_proba(X_val)[:, 1]
        y_pred = (y_prob >= threshold).astype(int)

        score = fbeta_score_custom(y_val, y_pred, beta=beta)
        scores.append(score)

        all_predictions.extend(y_pred)
        all_true_values.extend(y_val)

    if len(scores) == 5:
        print("\n=== 交差検証全体の評価 ===")
        metrics = calculate_detailed_metrics(all_true_values, all_predictions)
        print("\nClassification Report:")
        print(classification_report(all_true_values, all_predictions))

    return np.mean(scores)

def objective(trial, X_vec, y, class_weights, beta=2):
    """Optunaの目的関数"""
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 128),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbosity": -1
    }
    threshold = trial.suggest_float("threshold", 0.1, 0.9)

    score = evaluate_model(X_vec, y, params, threshold, class_weights, beta=beta)
    return score

def train_final_model(df, beta_value=2, n_trials=50):
    """完全なトレーニングパイプライン"""
    # データの準備
    X_vec, y, vectorizer, class_weights = prepare_text_data(df)

    print(f"データセットの形状: {X_vec.shape}")
    print(f"クラスの分布:\n{y.value_counts(normalize=True)}")
    print(f"クラスの重み: {class_weights}")

    # ハイパーパラメータの最適化
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective(trial, X_vec, y, class_weights, beta=beta_value),
        n_trials=n_trials
    )

    best_params = study.best_trial.params
    threshold = best_params.pop("threshold")

    print("\nBest parameters:", best_params)
    print("Best threshold:", threshold)

    final_score = evaluate_model(X_vec, y, best_params, threshold, class_weights, beta=beta_value)
    print(f"\nFinal Mean F{beta_value} Score:", final_score)

    # 最終モデルの学習
    final_model = LGBMClassifier(
        **best_params,
        random_state=42,
        class_weight=class_weights
    )
    final_model.fit(X_vec, y)

    # 最終評価
    print("\n=== 最終モデルの評価（全データ） ===")
    y_prob = final_model.predict_proba(X_vec)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)
    final_metrics = calculate_detailed_metrics(y, y_pred)

    return final_model, vectorizer, threshold, best_params

def predict_new_text(text, model, vectorizer, threshold):
    """新しいテキストの予測"""
    X_new = vectorizer.transform([text])
    prob = model.predict_proba(X_new)[0, 1]
    prediction = prob >= threshold
    return prediction, prob

In [ ]:
# 使用例
if __name__ == "__main__":
    # データの読み込み
    #df = pd.read_csv('your_data.csv')  # tiab列とincluded列を含むデータ

    # モデルのトレーニング
    model, vectorizer, threshold, best_params = train_final_model(
        rayyan_decided_df,
        beta_value=4,  # より高い値で再現率を重視
        n_trials=50
    )

In [ ]:
import joblib

# 日時をstringsで
timestamp = time.strftime('%Y%m%d%H%M%S')

# モデルと関連オブジェクトの保存
model_data = {
    'model': model,
    'vectorizer': vectorizer,
    'threshold': threshold,
    'best_params': best_params
}
joblib.dump(model_data, f'{timestamp}_v2_model_data.joblib')
print("\nモデルと関連オブジェクトを保存しました。")

In [ ]:
models = glob.glob('*.joblib')
print(models)

In [ ]:
# 保存したモデルデータをロード
model_data = joblib.load(models[2])

# 各オブジェクトの取り出し
loaded_model = model_data['model']
loaded_vectorizer = model_data['vectorizer']
loaded_threshold = model_data['threshold']
loaded_best_params = model_data['best_params']

print("モデルと関連オブジェクトをロードしました。")

In [ ]:
import glob
import joblib

# モデルファイルを検索
models = glob.glob('*.joblib')
print(f"見つかったモデルファイル: {models}")

if not models:
    print("joblib ファイルが見つかりません。")
else:
    # インデックスを修正 - 最初の要素を使用
    model_file = models[0]
    print(f"使用するモデルファイル: {model_file}")
    
    try:
        # 保存したモデルデータをロード
        model_data = joblib.load(model_file)
        
        # 各オブジェクトの取り出し
        loaded_model = model_data['model']
        loaded_vectorizer = model_data['vectorizer']
        loaded_threshold = model_data['threshold']
        loaded_best_params = model_data['best_params']
        
        print("モデルと関連オブジェクトをロードしました。")
        print(f"モデルの種類: {type(loaded_model).__name__}")
        print(f"最適な閾値: {loaded_threshold}")
        print(f"最適なパラメータ: {loaded_best_params}")
    except Exception as e:
        print(f"モデルのロード中にエラーが発生しました: {str(e)}")
        
        # ファイルの内容を確認
        try:
            test_load = joblib.load(model_file)
            print(f"ファイルには次のキーが含まれています: {list(test_load.keys()) if isinstance(test_load, dict) else '辞書ではありません'}")
        except:
            print("ファイルの内容を確認できませんでした。")

In [ ]:
def predict_and_summarize(df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold):
    # "tiab"列の作成 (DataFrame dfに対して適用)
    df["tiab"] = df.apply(create_json_text, axis=1)

    # テキストデータの前処理とベクトル化
    X_new = loaded_vectorizer.transform(df["tiab"].fillna(''))

    print(f"新しいデータの形状: {X_new.shape}")

    # 予測確率の取得
    y_prob_new = loaded_model.predict_proba(X_new)[:, 1]

    # 閾値を適用してクラスラベルを決定
    y_pred_new = (y_prob_new >= loaded_threshold).astype(int)

    # 予測結果をデータフレームに追加
    df['prediction'] = y_pred_new
    df['probability'] = y_prob_new

    print("予測を実行し、結果をデータフレームに追加しました。")

    # 予測結果の表示
    print("\n=== 予測結果 ===")
    print(df[['tiab', 'prediction', 'probability']])

    # クラス分布の確認
    class_dist = df['prediction'].value_counts()
    print("\nクラスの分布:")
    print(class_dist)

    # 確率の統計情報
    prob_stats = df['probability'].describe()
    print("\n予測確率の統計情報:")
    print(prob_stats)

    # 必要に応じて、更新後のdfや統計情報を返す
    return df, class_dist, prob_stats

In [ ]:
# BibTeXエントリをDataFrameに変換する関数
def convert_entries_to_df(entries):
    # 各エントリからフィールドを抽出
    formatted_entries = []
    for entry in entries:
        # 各フィールドを取得（存在しない場合は空文字を返す）
        title = entry.get('title', '').strip('{}')
        
        # 著者の整形
        authors = entry.get('author', '').replace('\n', ' ').strip('{}')
        
        # 掲載誌/会議名
        journal = entry.get('journal', entry.get('booktitle', ''))
        
        # 出版年
        year = entry.get('year', '')
        
        # DOI
        doi = entry.get('doi', '')
        
        # Abstract
        abstract = entry.get('abstract', '').strip('{}')
        
        # PMID
        pmid = entry.get('pmid', '')
        
        formatted_entries.append({
            'Title': title,
            'Authors': authors,
            'Abstract': abstract,
            'Journal': journal,
            'Year': year,
            'DOI': doi,
            'PMID': pmid
        })
    
    # DataFrameに変換
    return pd.DataFrame(formatted_entries)

# remaining_entriesをDataFrameに変換
print(f"残りの{len(remaining_entries)}件のエントリをDataFrameに変換中...")
remaining_df = convert_entries_to_df(remaining_entries)

# tiab列を作る関数（既に定義済み）
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

# 機械学習フィルターを適用
print("機械学習フィルターを適用中...")
filtered_df, class_distribution, probability_stats = predict_and_summarize(
    remaining_df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)

# 「含まれる」と予測されたエントリのみを抽出
included_df = filtered_df[filtered_df['prediction'] == 1]
print(f"\n「含まれる」と予測されたエントリ: {len(included_df)}件")


In [ ]:
# 検索したいPMIDのリスト（文字列から数字だけを抽出）
pmid_list_raw = """1888246[pmid] or 2393205[pmid] or 16586387[pmid] or 9395366[pmid] or 8263576[pmid] or 14630621[pmid] or 
15482728[pmid] or 18486413[pmid] or 19555286[pmid] or 20586653[pmid] or 20299634[pmid] or 20557905[pmid] or 
21722666[pmid] or 23808716[pmid] or 24856736[pmid] or 18486413[pmid] or 20557905[pmid] or 21083886[pmid] or 
21684227[pmid] or 22463870[pmid] or 24856736[pmid] or 26656373[pmid] or 26246024[pmid] or 28714287[pmid] or 
32291611[pmid] or 27023336[pmid] or 33740571[pmid] or 33810788[pmid] or 34992000[pmid] or 34032112[pmid] or 
34487306[pmid] or 32796184[pmid] or 31581716[pmid] or 35054269[pmid] or 32646430[pmid] or 37620774[pmid] or 
35351003[pmid] or 33658812[pmid] or 34484472[pmid] or 38396484[pmid] or 30111827[pmid] or 34428529[pmid] or 
29661634[pmid] or 37519767[pmid] or 34309570[pmid] or 33347714[pmid] or 34162572[pmid] or 37719647[pmid] or 
34998038[pmid] or 37598221[pmid] or 34983764[pmid] or 37244761[pmid] or 35853298[pmid] or 36292187[pmid] or 
37562317[pmid] or 31912208[pmid] or 30420241[pmid] or 34209759[pmid] or 29486341[pmid] or 34490306[pmid]"""

# 数字のみを抽出
pmid_list = re.findall(r'(\d+)', pmid_list_raw)

# 重複を除去（リスト内に重複PMIDがある）
pmid_list = list(set(pmid_list))

print(f"検索するPMID数: {len(pmid_list)}")

# included_dfのPMID列を確認
if 'PMID' not in included_df.columns:
    print("警告: included_dfにPMID列が見つかりません。確認できません。")
else:
    # 結果を格納する辞書
    filtered_results = {
        'found': [],    # フィルター結果に見つかったPMID
        'not_found': [] # フィルター結果に見つからなかったPMID
    }
    
    # PMIDの存在確認（直接マッチング）
    for target_pmid in pmid_list:
        # PMID列で直接一致を確認
        mask = included_df['PMID'].astype(str) == target_pmid
        if mask.any():
            # 見つかった場合、その行の情報を保存
            matching_rows = included_df[mask]
            for _, row in matching_rows.iterrows():
                filtered_results['found'].append({
                    'pmid': target_pmid,
                    'title': row['Title'],
                    'probability': row['probability'] if 'probability' in included_df.columns else 'N/A'
                })
        else:
            # 見つからなかった場合
            filtered_results['not_found'].append(target_pmid)
    
    # 結果を表示
    print("\n=== 機械学習フィルター結果内のPMID検索結果 ===")
    print(f"フィルター後のエントリ数: {len(included_df)}")
    print(f"検索したPMID数: {len(pmid_list)}")
    print(f"フィルター結果内に見つかったPMID数: {len(filtered_results['found'])}")
    print(f"フィルター結果内に見つからなかったPMID数: {len(filtered_results['not_found'])}")
    
    # 見つかったPMIDの詳細を表示
    if filtered_results['found']:
        print("\n=== フィルター結果内に見つかったPMIDの詳細 ===")
        for i, entry in enumerate(filtered_results['found'], 1):
            print(f"\n--- {i}. PMID: {entry['pmid']} ---")
            print(f"タイトル: {entry['title']}")
            print(f"予測確率: {entry['probability']}")
    
    # 見つからなかったPMIDのリスト
    if filtered_results['not_found']:
        print("\n=== フィルター結果内に見つからなかったPMID ===")
        # 10個ずつ表示
        for i in range(0, len(filtered_results['not_found']), 10):
            chunk = filtered_results['not_found'][i:i+10]
            print(", ".join(chunk))
    
    # 結果をファイルに保存
    with open('filtered_pmid_results.txt', 'w', encoding='utf-8') as f:
        f.write("=== フィルター結果内に見つかったPMID ===\n")
        for entry in filtered_results['found']:
            f.write(f"PMID: {entry['pmid']}, タイトル: {entry['title']}, 予測確率: {entry['probability']}\n")
        
        f.write("\n=== フィルター結果内に見つからなかったPMID ===\n")
        f.write(", ".join(filtered_results['not_found']))
    
    print("\n検索結果をfiltered_pmid_results.txtに保存しました。")

    # PMIDが見つからなかった場合の分析
    if filtered_results['not_found']:
        print("\n=== 見つからなかったPMIDの再検索（部分一致） ===")
        print("PMIDが部分的に含まれている可能性があるエントリを検索します...")
        
        partial_matches = []
        for target_pmid in filtered_results['not_found']:
            # PMID列が文字列であることを確認し、部分一致を検索
            if 'PMID' in included_df.columns:
                mask = included_df['PMID'].astype(str).str.contains(target_pmid, na=False, regex=False)
                matching_rows = included_df[mask]
                
                if not matching_rows.empty:
                    for _, row in matching_rows.iterrows():
                        partial_matches.append({
                            'pmid': target_pmid,
                            'found_in': row['PMID'],
                            'title': row['Title']
                        })
        
        if partial_matches:
            print(f"部分一致するPMIDが{len(partial_matches)}件見つかりました:")
            for i, match in enumerate(partial_matches, 1):
                print(f"{i}. 検索PMID: {match['pmid']}, 見つかったPMID: {match['found_in']}, タイトル: {match['title']}")
        else:
            print("部分一致するPMIDは見つかりませんでした。")

In [ ]:
from datetime import datetime

In [ ]:
import random

In [ ]:
# 現在の日時を取得（ファイル名用）
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# ステップ1: included_dfから500件をランダムに抽出
print(f"included_dfには{len(included_df)}件のエントリがあります")

# ランダム抽出のシード値を設定（再現性のため）
random.seed(42)

# 500件をランダムに抽出
sample_size = min(500, len(included_df))
sampled_indices = random.sample(range(len(included_df)), sample_size)
sampled_df = included_df.iloc[sampled_indices].copy()

# 残りのエントリを取得
remaining_indices = [i for i in range(len(included_df)) if i not in sampled_indices]
remaining_df = included_df.iloc[remaining_indices].copy()

print(f"ランダムに{len(sampled_df)}件抽出しました")
print(f"残りのエントリ数: {len(remaining_df)}")

# ステップ2: 抽出した500件をRayyan用のCSVとして保存
# Rayyan形式に合わせて必要なカラムを選択
rayyan_columns = ['Title', 'Authors', 'Abstract', 'Journal', 'Year', 'DOI', 'PMID']
rayyan_df = sampled_df[rayyan_columns].copy()

# Rayyan用のCSVファイル名
rayyan_csv_file = f"rayyan_ml_sample_{sample_size}_{timestamp}.csv"
rayyan_df.to_csv(rayyan_csv_file, index=False, encoding='utf-8')
print(f"Rayyan用に{len(rayyan_df)}件のエントリを {rayyan_csv_file} に保存しました")

# ステップ3: 残りのエントリに対して新しい機械学習フィルターを適用する準備
# さらにフィルタリングするためにタイトルとアブストラクトを組み合わせた列を作成
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

# 機械学習用のtiab列を生成
print("機械学習用のtiab列を生成中...")
remaining_df['tiab'] = remaining_df.apply(create_json_text, axis=1)

# 残りのエントリをCSVとして保存
remaining_csv_file = f"remaining_ml_entries_{timestamp}.csv"
remaining_df.to_csv(remaining_csv_file, index=False, encoding='utf-8')
print(f"残りの{len(remaining_df)}件のエントリを {remaining_csv_file} に保存しました")

In [ ]:
# 2nd ml
# Load the CSV file into a DataFrame
rayyan_decided_df = pd.read_csv('rayyan_decided_2nd500.csv',encoding='cp932')

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(rayyan_decided_df.head())

In [ ]:
# rayyan_decided_dfに対してincludedフラグを作成
# 'notes'列に'Included'という文字列が含まれていればTrue
rayyan_decided_df['included'] = rayyan_decided_df['notes'].astype(str).str.contains('Included')

# 結果の集計を表示
print(rayyan_decided_df['included'].value_counts())

# 不整合（'included'がTrueなのに'notes'に'Excluded'が含まれる）の件数
inconsistency_count = rayyan_decided_df[rayyan_decided_df["included"] == True]["notes"].str.contains("Excluded").sum()
print(f'不整合の件数: {inconsistency_count}')

# タイトルと抄録を組み合わせた新しい列'tiab'を作成
def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

# 'tiab'列を作成
rayyan_decided_df['tiab'] = rayyan_decided_df.apply(create_json_text, axis=1)

# 最初の数行を表示して確認
print("\n最初の3行のデータ:")
print(rayyan_decided_df[['title', 'abstract', 'notes', 'included', 'tiab']].head(3))

# 必要に応じてCSVとして保存
# rayyan_decided_df.to_csv('rayyan_processed.csv', index=False)

In [ ]:
# 使用例
if __name__ == "__main__":
    # データの読み込み
    #df = pd.read_csv('your_data.csv')  # tiab列とincluded列を含むデータ

    # モデルのトレーニング
    model, vectorizer, threshold, best_params = train_final_model(
        rayyan_decided_df,
        beta_value=4,  # より高い値で再現率を重視
        n_trials=50
    )

In [ ]:
# 日時をstringsで
timestamp = time.strftime('%Y%m%d%H%M%S')

# モデルと関連オブジェクトの保存
model_data = {
    'model': model,
    'vectorizer': vectorizer,
    'threshold': threshold,
    'best_params': best_params
}
joblib.dump(model_data, f'{timestamp}_v2_model_data.joblib')
print("\nモデルと関連オブジェクトを保存しました。")

In [ ]:
models = glob.glob('*.joblib')
print(models)

In [ ]:
import glob
import joblib

# モデルファイルを検索
models = glob.glob('20250306154131_v2_model_data.joblib')
print(f"見つかったモデルファイル: {models}")

if not models:
    print("joblib ファイルが見つかりません。")
else:
    # インデックスを修正 - 最初の要素を使用
    model_file = models[0]
    print(f"使用するモデルファイル: {model_file}")
    
    try:
        # 保存したモデルデータをロード
        model_data = joblib.load(model_file)
        
        # 各オブジェクトの取り出し
        loaded_model = model_data['model']
        loaded_vectorizer = model_data['vectorizer']
        loaded_threshold = model_data['threshold']
        loaded_best_params = model_data['best_params']
        
        print("モデルと関連オブジェクトをロードしました。")
        print(f"モデルの種類: {type(loaded_model).__name__}")
        print(f"最適な閾値: {loaded_threshold}")
        print(f"最適なパラメータ: {loaded_best_params}")
    except Exception as e:
        print(f"モデルのロード中にエラーが発生しました: {str(e)}")
        
        # ファイルの内容を確認
        try:
            test_load = joblib.load(model_file)
            print(f"ファイルには次のキーが含まれています: {list(test_load.keys()) if isinstance(test_load, dict) else '辞書ではありません'}")
        except:
            print("ファイルの内容を確認できませんでした。")

In [ ]:
# 残っている文献をcsvにしてあったので読み込む
remaining_df = pd.read_csv('remaining_ml_entries_20250303142313.csv')

# tiab列を作る関数（既に定義済み）
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

# 機械学習フィルターを適用
print("機械学習フィルターを適用中...")
filtered_df, class_distribution, probability_stats = predict_and_summarize(
    remaining_df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)

# 「含まれる」と予測されたエントリのみを抽出
included_df = filtered_df[filtered_df['prediction'] == 1]
print(f"\n「含まれる」と予測されたエントリ: {len(included_df)}件")

In [ ]:
# 検索したいPMIDのリスト（文字列から数字だけを抽出）
pmid_list_raw = """1888246[pmid] or 2393205[pmid] or 16586387[pmid] or 9395366[pmid] or 8263576[pmid] or 14630621[pmid] or 
15482728[pmid] or 18486413[pmid] or 19555286[pmid] or 20586653[pmid] or 20299634[pmid] or 20557905[pmid] or 
21722666[pmid] or 23808716[pmid] or 24856736[pmid] or 18486413[pmid] or 20557905[pmid] or 21083886[pmid] or 
21684227[pmid] or 22463870[pmid] or 24856736[pmid] or 26656373[pmid] or 26246024[pmid] or 28714287[pmid] or 
32291611[pmid] or 27023336[pmid] or 33740571[pmid] or 33810788[pmid] or 34992000[pmid] or 34032112[pmid] or 
34487306[pmid] or 32796184[pmid] or 31581716[pmid] or 35054269[pmid] or 32646430[pmid] or 37620774[pmid] or 
35351003[pmid] or 33658812[pmid] or 34484472[pmid] or 38396484[pmid] or 30111827[pmid] or 34428529[pmid] or 
29661634[pmid] or 37519767[pmid] or 34309570[pmid] or 33347714[pmid] or 34162572[pmid] or 37719647[pmid] or 
34998038[pmid] or 37598221[pmid] or 34983764[pmid] or 37244761[pmid] or 35853298[pmid] or 36292187[pmid] or 
37562317[pmid] or 31912208[pmid] or 30420241[pmid] or 34209759[pmid] or 29486341[pmid] or 34490306[pmid]"""

# 数字のみを抽出
pmid_list = re.findall(r'(\d+)', pmid_list_raw)

# 重複を除去（リスト内に重複PMIDがある）
pmid_list = list(set(pmid_list))

print(f"検索するPMID数: {len(pmid_list)}")

# included_dfのPMID列を確認
if 'PMID' not in included_df.columns:
    print("警告: included_dfにPMID列が見つかりません。確認できません。")
else:
    # 結果を格納する辞書
    filtered_results = {
        'found': [],    # フィルター結果に見つかったPMID
        'not_found': [] # フィルター結果に見つからなかったPMID
    }
    
    # PMIDの存在確認（直接マッチング）
    for target_pmid in pmid_list:
        # PMID列で直接一致を確認
        mask = included_df['PMID'].astype(str) == target_pmid
        if mask.any():
            # 見つかった場合、その行の情報を保存
            matching_rows = included_df[mask]
            for _, row in matching_rows.iterrows():
                filtered_results['found'].append({
                    'pmid': target_pmid,
                    'title': row['Title'],
                    'probability': row['probability'] if 'probability' in included_df.columns else 'N/A'
                })
        else:
            # 見つからなかった場合
            filtered_results['not_found'].append(target_pmid)
    
    # 結果を表示
    print("\n=== 機械学習フィルター結果内のPMID検索結果 ===")
    print(f"フィルター後のエントリ数: {len(included_df)}")
    print(f"検索したPMID数: {len(pmid_list)}")
    print(f"フィルター結果内に見つかったPMID数: {len(filtered_results['found'])}")
    print(f"フィルター結果内に見つからなかったPMID数: {len(filtered_results['not_found'])}")
    
    # 見つかったPMIDの詳細を表示
    if filtered_results['found']:
        print("\n=== フィルター結果内に見つかったPMIDの詳細 ===")
        for i, entry in enumerate(filtered_results['found'], 1):
            print(f"\n--- {i}. PMID: {entry['pmid']} ---")
            print(f"タイトル: {entry['title']}")
            print(f"予測確率: {entry['probability']}")
    
    # 見つからなかったPMIDのリスト
    if filtered_results['not_found']:
        print("\n=== フィルター結果内に見つからなかったPMID ===")
        # 10個ずつ表示
        for i in range(0, len(filtered_results['not_found']), 10):
            chunk = filtered_results['not_found'][i:i+10]
            print(", ".join(chunk))
    
    # 結果をファイルに保存
    with open('filtered_pmid_results.txt', 'w', encoding='utf-8') as f:
        f.write("=== フィルター結果内に見つかったPMID ===\n")
        for entry in filtered_results['found']:
            f.write(f"PMID: {entry['pmid']}, タイトル: {entry['title']}, 予測確率: {entry['probability']}\n")
        
        f.write("\n=== フィルター結果内に見つからなかったPMID ===\n")
        f.write(", ".join(filtered_results['not_found']))
    
    print("\n検索結果をfiltered_pmid_results.txtに保存しました。")

    # PMIDが見つからなかった場合の分析
    if filtered_results['not_found']:
        print("\n=== 見つからなかったPMIDの再検索（部分一致） ===")
        print("PMIDが部分的に含まれている可能性があるエントリを検索します...")
        
        partial_matches = []
        for target_pmid in filtered_results['not_found']:
            # PMID列が文字列であることを確認し、部分一致を検索
            if 'PMID' in included_df.columns:
                mask = included_df['PMID'].astype(str).str.contains(target_pmid, na=False, regex=False)
                matching_rows = included_df[mask]
                
                if not matching_rows.empty:
                    for _, row in matching_rows.iterrows():
                        partial_matches.append({
                            'pmid': target_pmid,
                            'found_in': row['PMID'],
                            'title': row['Title']
                        })
        
        if partial_matches:
            print(f"部分一致するPMIDが{len(partial_matches)}件見つかりました:")
            for i, match in enumerate(partial_matches, 1):
                print(f"{i}. 検索PMID: {match['pmid']}, 見つかったPMID: {match['found_in']}, タイトル: {match['title']}")
        else:
            print("部分一致するPMIDは見つかりませんでした。")

In [ ]:
# 現在の日時を取得（ファイル名用）
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# ステップ1: included_dfから500件をランダムに抽出
print(f"included_dfには{len(included_df)}件のエントリがあります")

# ランダム抽出のシード値を設定（再現性のため）
random.seed(42)

# 500件をランダムに抽出
sample_size = min(500, len(included_df))
sampled_indices = random.sample(range(len(included_df)), sample_size)
sampled_df = included_df.iloc[sampled_indices].copy()

# 残りのエントリを取得
remaining_indices = [i for i in range(len(included_df)) if i not in sampled_indices]
remaining_df = included_df.iloc[remaining_indices].copy()

print(f"ランダムに{len(sampled_df)}件抽出しました")
print(f"残りのエントリ数: {len(remaining_df)}")

# ステップ2: 抽出した500件をRayyan用のCSVとして保存
# Rayyan形式に合わせて必要なカラムを選択
rayyan_columns = ['Title', 'Authors', 'Abstract', 'Journal', 'Year', 'DOI', 'PMID']
rayyan_df = sampled_df[rayyan_columns].copy()

# Rayyan用のCSVファイル名
rayyan_csv_file = f"rayyan_ml_sample_{sample_size}_{timestamp}.csv"
rayyan_df.to_csv(rayyan_csv_file, index=False, encoding='utf-8')
print(f"Rayyan用に{len(rayyan_df)}件のエントリを {rayyan_csv_file} に保存しました")

# ステップ3: 残りのエントリに対して新しい機械学習フィルターを適用する準備
# さらにフィルタリングするためにタイトルとアブストラクトを組み合わせた列を作成
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

# 機械学習用のtiab列を生成
print("機械学習用のtiab列を生成中...")
remaining_df['tiab'] = remaining_df.apply(create_json_text, axis=1)

# 残りのエントリをCSVとして保存
remaining_csv_file = f"remaining_ml_entries_{timestamp}.csv"
remaining_df.to_csv(remaining_csv_file, index=False, encoding='utf-8')
print(f"残りの{len(remaining_df)}件のエントリを {remaining_csv_file} に保存しました")

In [ ]:
# 3rd ml
# Load the CSV file into a DataFrame
rayyan_decided_df = pd.read_csv('rayyan_decided_3rd500.csv',encoding='cp932')

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(rayyan_decided_df.head())

In [ ]:
# rayyan_decided_dfに対してincludedフラグを作成
# 'notes'列に'Included'という文字列が含まれていればTrue
rayyan_decided_df['included'] = rayyan_decided_df['notes'].astype(str).str.contains('Included')

# 結果の集計を表示
print(rayyan_decided_df['included'].value_counts())

# 不整合（'included'がTrueなのに'notes'に'Excluded'が含まれる）の件数
inconsistency_count = rayyan_decided_df[rayyan_decided_df["included"] == True]["notes"].str.contains("Excluded").sum()
print(f'不整合の件数: {inconsistency_count}')

# タイトルと抄録を組み合わせた新しい列'tiab'を作成
def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

# 'tiab'列を作成
rayyan_decided_df['tiab'] = rayyan_decided_df.apply(create_json_text, axis=1)

# 最初の数行を表示して確認
print("\n最初の3行のデータ:")
print(rayyan_decided_df[['title', 'abstract', 'notes', 'included', 'tiab']].head(3))

# 必要に応じてCSVとして保存
# rayyan_decided_df.to_csv('rayyan_processed.csv', index=False)

In [ ]:
# 使用例
if __name__ == "__main__":
    # データの読み込み
    #df = pd.read_csv('your_data.csv')  # tiab列とincluded列を含むデータ

    # モデルのトレーニング
    model, vectorizer, threshold, best_params = train_final_model(
        rayyan_decided_df,
        beta_value=4,  # より高い値で再現率を重視
        n_trials=50
    )

In [ ]:
# 日時をstringsで
timestamp = time.strftime('%Y%m%d%H%M%S')

# モデルと関連オブジェクトの保存
model_data = {
    'model': model,
    'vectorizer': vectorizer,
    'threshold': threshold,
    'best_params': best_params
}
joblib.dump(model_data, f'{timestamp}_v2_model_data.joblib')
print("\nモデルと関連オブジェクトを保存しました。")

In [ ]:
models = glob.glob('*.joblib')
print(models)

In [ ]:
import glob
import joblib

# モデルファイルを検索
models = glob.glob('20250311144512_v2_model_data.joblib')
print(f"見つかったモデルファイル: {models}")

if not models:
    print("joblib ファイルが見つかりません。")
else:
    # インデックスを修正 - 最初の要素を使用
    model_file = models[0]
    print(f"使用するモデルファイル: {model_file}")
    
    try:
        # 保存したモデルデータをロード
        model_data = joblib.load(model_file)
        
        # 各オブジェクトの取り出し
        loaded_model = model_data['model']
        loaded_vectorizer = model_data['vectorizer']
        loaded_threshold = model_data['threshold']
        loaded_best_params = model_data['best_params']
        
        print("モデルと関連オブジェクトをロードしました。")
        print(f"モデルの種類: {type(loaded_model).__name__}")
        print(f"最適な閾値: {loaded_threshold}")
        print(f"最適なパラメータ: {loaded_best_params}")
    except Exception as e:
        print(f"モデルのロード中にエラーが発生しました: {str(e)}")
        
        # ファイルの内容を確認
        try:
            test_load = joblib.load(model_file)
            print(f"ファイルには次のキーが含まれています: {list(test_load.keys()) if isinstance(test_load, dict) else '辞書ではありません'}")
        except:
            print("ファイルの内容を確認できませんでした。")

In [ ]:
# 残っている文献をcsvにしてあったので読み込む
remaining_df = pd.read_csv('remaining_ml_entries_20250306155821.csv')

# tiab列を作る関数（既に定義済み）
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

# 機械学習フィルターを適用
print("機械学習フィルターを適用中...")
filtered_df, class_distribution, probability_stats = predict_and_summarize(
    remaining_df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)

# 「含まれる」と予測されたエントリのみを抽出
included_df = filtered_df[filtered_df['prediction'] == 1]
print(f"\n「含まれる」と予測されたエントリ: {len(included_df)}件")

In [ ]:
# 検索したいPMIDのリスト（文字列から数字だけを抽出）
pmid_list_raw = """1888246[pmid] or 2393205[pmid] or 16586387[pmid] or 9395366[pmid] or 8263576[pmid] or 14630621[pmid] or 
15482728[pmid] or 18486413[pmid] or 19555286[pmid] or 20586653[pmid] or 20299634[pmid] or 20557905[pmid] or 
21722666[pmid] or 23808716[pmid] or 24856736[pmid] or 18486413[pmid] or 20557905[pmid] or 21083886[pmid] or 
21684227[pmid] or 22463870[pmid] or 24856736[pmid] or 26656373[pmid] or 26246024[pmid] or 28714287[pmid] or 
32291611[pmid] or 27023336[pmid] or 33740571[pmid] or 33810788[pmid] or 34992000[pmid] or 34032112[pmid] or 
34487306[pmid] or 32796184[pmid] or 31581716[pmid] or 35054269[pmid] or 32646430[pmid] or 37620774[pmid] or 
35351003[pmid] or 33658812[pmid] or 34484472[pmid] or 38396484[pmid] or 30111827[pmid] or 34428529[pmid] or 
29661634[pmid] or 37519767[pmid] or 34309570[pmid] or 33347714[pmid] or 34162572[pmid] or 37719647[pmid] or 
34998038[pmid] or 37598221[pmid] or 34983764[pmid] or 37244761[pmid] or 35853298[pmid] or 36292187[pmid] or 
37562317[pmid] or 31912208[pmid] or 30420241[pmid] or 34209759[pmid] or 29486341[pmid] or 34490306[pmid]"""

# 数字のみを抽出
pmid_list = re.findall(r'(\d+)', pmid_list_raw)

# 重複を除去（リスト内に重複PMIDがある）
pmid_list = list(set(pmid_list))

print(f"検索するPMID数: {len(pmid_list)}")

# included_dfのPMID列を確認
if 'PMID' not in included_df.columns:
    print("警告: included_dfにPMID列が見つかりません。確認できません。")
else:
    # 結果を格納する辞書
    filtered_results = {
        'found': [],    # フィルター結果に見つかったPMID
        'not_found': [] # フィルター結果に見つからなかったPMID
    }
    
    # PMIDの存在確認（直接マッチング）
    for target_pmid in pmid_list:
        # PMID列で直接一致を確認
        mask = included_df['PMID'].astype(str) == target_pmid
        if mask.any():
            # 見つかった場合、その行の情報を保存
            matching_rows = included_df[mask]
            for _, row in matching_rows.iterrows():
                filtered_results['found'].append({
                    'pmid': target_pmid,
                    'title': row['Title'],
                    'probability': row['probability'] if 'probability' in included_df.columns else 'N/A'
                })
        else:
            # 見つからなかった場合
            filtered_results['not_found'].append(target_pmid)
    
    # 結果を表示
    print("\n=== 機械学習フィルター結果内のPMID検索結果 ===")
    print(f"フィルター後のエントリ数: {len(included_df)}")
    print(f"検索したPMID数: {len(pmid_list)}")
    print(f"フィルター結果内に見つかったPMID数: {len(filtered_results['found'])}")
    print(f"フィルター結果内に見つからなかったPMID数: {len(filtered_results['not_found'])}")
    
    # 見つかったPMIDの詳細を表示
    if filtered_results['found']:
        print("\n=== フィルター結果内に見つかったPMIDの詳細 ===")
        for i, entry in enumerate(filtered_results['found'], 1):
            print(f"\n--- {i}. PMID: {entry['pmid']} ---")
            print(f"タイトル: {entry['title']}")
            print(f"予測確率: {entry['probability']}")
    
    # 見つからなかったPMIDのリスト
    if filtered_results['not_found']:
        print("\n=== フィルター結果内に見つからなかったPMID ===")
        # 10個ずつ表示
        for i in range(0, len(filtered_results['not_found']), 10):
            chunk = filtered_results['not_found'][i:i+10]
            print(", ".join(chunk))
    
    # 結果をファイルに保存
    with open('filtered_pmid_results.txt', 'w', encoding='utf-8') as f:
        f.write("=== フィルター結果内に見つかったPMID ===\n")
        for entry in filtered_results['found']:
            f.write(f"PMID: {entry['pmid']}, タイトル: {entry['title']}, 予測確率: {entry['probability']}\n")
        
        f.write("\n=== フィルター結果内に見つからなかったPMID ===\n")
        f.write(", ".join(filtered_results['not_found']))
    
    print("\n検索結果をfiltered_pmid_results.txtに保存しました。")

    # PMIDが見つからなかった場合の分析
    if filtered_results['not_found']:
        print("\n=== 見つからなかったPMIDの再検索（部分一致） ===")
        print("PMIDが部分的に含まれている可能性があるエントリを検索します...")
        
        partial_matches = []
        for target_pmid in filtered_results['not_found']:
            # PMID列が文字列であることを確認し、部分一致を検索
            if 'PMID' in included_df.columns:
                mask = included_df['PMID'].astype(str).str.contains(target_pmid, na=False, regex=False)
                matching_rows = included_df[mask]
                
                if not matching_rows.empty:
                    for _, row in matching_rows.iterrows():
                        partial_matches.append({
                            'pmid': target_pmid,
                            'found_in': row['PMID'],
                            'title': row['Title']
                        })
        
        if partial_matches:
            print(f"部分一致するPMIDが{len(partial_matches)}件見つかりました:")
            for i, match in enumerate(partial_matches, 1):
                print(f"{i}. 検索PMID: {match['pmid']}, 見つかったPMID: {match['found_in']}, タイトル: {match['title']}")
        else:
            print("部分一致するPMIDは見つかりませんでした。")

In [ ]:
# 現在の日時を取得（ファイル名用）
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# ステップ1: included_dfから500件をランダムに抽出
print(f"included_dfには{len(included_df)}件のエントリがあります")

# ランダム抽出のシード値を設定（再現性のため）
random.seed(42)

# 500件をランダムに抽出
sample_size = min(500, len(included_df))
sampled_indices = random.sample(range(len(included_df)), sample_size)
sampled_df = included_df.iloc[sampled_indices].copy()

# 残りのエントリを取得
remaining_indices = [i for i in range(len(included_df)) if i not in sampled_indices]
remaining_df = included_df.iloc[remaining_indices].copy()

print(f"ランダムに{len(sampled_df)}件抽出しました")
print(f"残りのエントリ数: {len(remaining_df)}")

# ステップ2: 抽出した500件をRayyan用のCSVとして保存
# Rayyan形式に合わせて必要なカラムを選択
rayyan_columns = ['Title', 'Authors', 'Abstract', 'Journal', 'Year', 'DOI', 'PMID']
rayyan_df = sampled_df[rayyan_columns].copy()

# Rayyan用のCSVファイル名
rayyan_csv_file = f"rayyan_ml_sample_{sample_size}_{timestamp}.csv"
rayyan_df.to_csv(rayyan_csv_file, index=False, encoding='utf-8')
print(f"Rayyan用に{len(rayyan_df)}件のエントリを {rayyan_csv_file} に保存しました")

# ステップ3: 残りのエントリに対して新しい機械学習フィルターを適用する準備
# さらにフィルタリングするためにタイトルとアブストラクトを組み合わせた列を作成
def create_json_text(row):
    return json.dumps({
        'title': str(row['Title']),
        'abstract': str(row['Abstract'])
    })

# 機械学習用のtiab列を生成
print("機械学習用のtiab列を生成中...")
remaining_df['tiab'] = remaining_df.apply(create_json_text, axis=1)

# 残りのエントリをCSVとして保存
remaining_csv_file = f"remaining_ml_entries_{timestamp}.csv"
remaining_df.to_csv(remaining_csv_file, index=False, encoding='utf-8')
print(f"残りの{len(remaining_df)}件のエントリを {remaining_csv_file} に保存しました")

In [ ]:
# 現在の日時を取得（ファイル名用）
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# included_dfのサイズを確認
print(f"included_dfには{len(included_df)}件のエントリがあります")

# ランダムシャッフルのシード値を設定（再現性のため）
random.seed(42)

# データフレームのインデックスをシャッフル
all_indices = list(range(len(included_df)))
random.shuffle(all_indices)

# 指定されたサイズで分割
group_sizes = [430, 430, 430, 430, 430, 420]

# 実際のデータサイズと確認
total_requested = sum(group_sizes)
if total_requested > len(included_df):
    print(f"警告: 要求された合計サイズ({total_requested})がデータサイズ({len(included_df)})を超えています")
    print("最後のグループのサイズを調整します")
    # 最後のグループのサイズを調整
    group_sizes[-1] = max(0, len(included_df) - sum(group_sizes[:-1]))

print(f"指定されたグループサイズ: {group_sizes}")
print(f"合計: {sum(group_sizes)}件")

# 各グループのインデックスを割り当て
group_indices = []
start_idx = 0

for i, size in enumerate(group_sizes):
    end_idx = start_idx + size
    if end_idx > len(all_indices):
        end_idx = len(all_indices)
    group_indices.append(all_indices[start_idx:end_idx])
    start_idx = end_idx
    
    # インデックスが足りなくなったら終了
    if start_idx >= len(all_indices):
        break

# 実際に作成されたグループ数
actual_groups = len(group_indices)
print(f"実際に作成されたグループ数: {actual_groups}")

# 各グループのサイズを確認
for i, indices in enumerate(group_indices, 1):
    print(f"グループ{i}のサイズ: {len(indices)}件")

# Rayyan形式用のカラム
rayyan_columns = ['Title', 'Authors', 'Abstract', 'Journal', 'Year', 'DOI', 'PMID']

# 各グループを別々のCSVファイルに保存
for i, indices in enumerate(group_indices, 1):
    # グループのデータフレームを作成
    group_df = included_df.iloc[indices].copy()
    
    # Rayyan用のカラムを選択
    if all(col in group_df.columns for col in rayyan_columns):
        rayyan_df = group_df[rayyan_columns].copy()
    else:
        # カラムが見つからない場合は、利用可能なすべてのカラムを使用
        rayyan_df = group_df.copy()
        print(f"警告: グループ{i}では一部のRayyan列が見つかりません。利用可能なすべての列を使用します。")
    
    # CSVファイル名
    csv_file = f"rayyan_group_{i}_of_{actual_groups}_{timestamp}.csv"
    
    # CSVとして保存
    rayyan_df.to_csv(csv_file, index=False, encoding='utf-8')
    print(f"グループ{i}（{len(rayyan_df)}件）を {csv_file} に保存しました")

print("\n処理完了！")

In [ ]:
import pandas as pd
import rispy

# ファイルパス
reference_path = "path/to/your/references.ris"
citation_path = "path/to/your/references.ris"

# RISファイルを読み込む関数
def read_ris_to_entries(file_path):
    with open(file_path, 'r', encoding='utf-8') as ris_file:
        entries = rispy.load(ris_file)
    return entries

# 両方のRISファイルを読み込む
reference_entries = read_ris_to_entries(reference_path)
citation_entries = read_ris_to_entries(citation_path)

# 両方のエントリを結合
all_entries = reference_entries + citation_entries

# エントリ数を表示
print(f"参照文献数: {len(reference_entries)}")
print(f"引用文献数: {len(citation_entries)}")
print(f"合計エントリ数: {len(all_entries)}")

# DataFrameに変換
df = pd.DataFrame(all_entries)

# DataFrameの基本情報を表示
print("\nDataFrameの列:")
print(df.columns.tolist())
print("\n最初の数行:")
print(df.head())

# BibTeXに変換したい場合は以下のようにします
# bibtex_db = bibtexparser.bibdatabase.BibDatabase()
# bibtex_db.entries = all_entries
# bibtex_str = bibtexparser.dumps(bibtex_db)
# with open('combined_references.bib', 'w', encoding='utf-8') as bibtex_file:
#     bibtex_file.write(bibtex_str)

In [ ]:
# 機械学習フィルターを適用
import json

def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

print("機械学習フィルターを適用中...")
filtered_df, class_distribution, probability_stats = predict_and_summarize(
    df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)

# 「含まれる」と予測されたエントリのみを抽出
included_df = filtered_df[filtered_df['prediction'] == 1]
print(f"\n「含まれる」と予測されたエントリ: {len(included_df)}件")

In [ ]:
import rispy
from datetime import datetime

# フィルタリング後のDataFrameをRISフォーマットで保存する
def save_to_ris(dataframe, output_path):
    # DataFrameの各行をrisエントリに変換
    ris_entries = []
    
    for _, row in dataframe.iterrows():
        # 行をdictに変換（NaNを除外）
        entry = row.dropna().to_dict()
        ris_entries.append(entry)
    
    # RISファイルとして保存
    with open(output_path, 'w', encoding='utf-8') as f:
        rispy.dump(ris_entries, f)
    
    print(f"保存完了: {output_path} ({len(ris_entries)}件のエントリ)")

# フィルタリング後のDataFrameをRISファイルとして保存
output_path = "path/to/your/references.ris"
save_to_ris(included_df, output_path)